In [ ]:
# Importação de bibliotecas externas necessárias
from deap import base, creator, tools, benchmarks
import deap.cma as cma

import copy
import random
import math
import time
import logging
import threading

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from scipy.integrate import odeint
from scipy.integrate import solve_ivp

import sys
import os

# Importação de módulos personalizados
sys.path.append(os.path.abspath("..")) 
from Modules.Helper import Helper
from Modules.Solvers import Solvers
from Modules.Plotters import Plotters
from Modules.Equation import Equation

In [ ]:
# Carrega os dados do sistema e define as condições iniciais
labels = ['A', 'B', 'C', 'D', 'E']
df, max_data = Helper.load_data(filename='Data/ECOLI_DATA.txt', labels=labels)
initial_conditions = np.array([df[label].iloc[0] for label in labels])
t_span = (df['t'].iloc[0], df['t'].iloc[-1])  # Intervalo de tempo para simulações
t_eval = np.array(df['t'])  # Ponto de avaliação dos dados temporais
original = np.array(df[labels]).T  # Dados originais para cálculo de erro

# Limites nos valores dos coeficientes tau, k e n 
bounds = {
    'tau': (0.1, 5.0),
    'k': (0.1, 2.0),
    'n': (0, 30.0)
}

IND_SIZE = 35  # Tamanho do indivíduo (número total de coeficientes)
GENERATION_LIMIT = 5000
NO_IMPROVEMENT_LIMIT = 50
HARD_SIGMA_INCREASE_FACTOR = 10
TOLERANCE = 1E-4

In [ ]:
# Representa um coeficiente com valor e limites
class Coefficient:
    def __init__(self, bounds_val):
        self.val = random.uniform(*bounds_val)  # Inicializa com valor aleatório dentro dos limites
        self.bounds_val = bounds_val
    
    def __repr__(self):
        return f"val={self.val}"

# Representa um coeficiente usado no CMA-ES com limites.
class CMACoefficient:
    def __init__(self, val, bounds_val):
        self.bounds_val = bounds_val
        self.val = self.limit_val(val)  # Ajusta o valor aos limites
    
    # Garante que o valor esteja dentro dos limites
    def limit_val(self, val):
        return max(self.bounds_val[0], min(val, self.bounds_val[1]))
    
    def __repr__(self):
        return f"val={self.val}"

# Representa um indivíduo contendo coeficientes e funções para manipulação
class Individual:
    def __init__(self, method='RK45', error='ABS'):
        # Estrutura do indivíduo contendo os coeficientes para cada variável
        self.coeffs = {
            'A': {
                'A': {'n': None, 'k': None},  # nAA & kAA
                'C': {'n': None, 'k': None},  # nAC & kAC
                'D': {'n': None, 'k': None},  # nAD & kAD
                'E': {'n': None, 'k': None},  # nAE & kAE
                'tau': None                   # tauA
            },
            'B': {
                'A': {'n': None, 'k': None}, # nBA & kBA
                'C': {'n': None, 'k': None}, # nBC & kBC
                'D': {'n': None, 'k': None}, # nBD & kBD
                'E': {'n': None, 'k': None}, # nBE & kBE
                'tau': None                  # tauB
            },
            'C': {
                'D': {'n': None, 'k': None}, # nCD & kCD
                'E': {'n': None, 'k': None}, # nCE & kCE
                'tau': None,                 # tauC
            },
            'D': {
                'C': {'n': None, 'k': None, '-': False}, # nDC & kDC
                'tau': None,                             # tauD
            },
            'E': {
                'A': {'n': None, 'k': None}, # nEA & kEA
                'B': {'n': None, 'k': None}, # nEB & kEB
                'C': {'n': None, 'k': None}, # nEC & kEC
                'E': {'n': None, 'k': None}, # nEE & kEE
                'tau': None,                 # tauD
            }
        }
        
        # Tamanho do indivíduo
        self.ind_size = IND_SIZE 
        
        # Fitness inicializado como infinito
        self.fitness = np.inf
        self.method = method
        self.error = error
     
    @staticmethod
    def list_to_ind(list_ind, method, error):
        i = 0
        ind = Individual(method=method, error=error)
        for key, label in ind.coeffs.items():
            label['tau'] = CMACoefficient(list_ind[i], bounds['tau'])
            i += 1
            for key, coeffs in label.items():
                if key != 'tau':
                    coeffs['n'] = CMACoefficient(list_ind[i], bounds['n'])
                    coeffs['k'] = CMACoefficient(list_ind[i+1], bounds['k'])
                    i += 2
        return ind
    
    def ind_to_list(self):
        ind_list = []
        for key, label in self.coeffs.items():
            ind_list.append(label['tau'].val)
            for key, coeffs in label.items():
                if key != 'tau':
                    ind_list.append(coeffs['n'].val)
                    ind_list.append(coeffs['k'].val)
        return ind_list
    
    @staticmethod
    def apply_bounds(population, method, error):
        for ind in population:
            list_ind = Individual.list_to_ind(ind, method, error)
            ind[:] = Individual.ind_to_list(list_ind)
    
    @staticmethod    
    def cma_evaluate(method, error, list_ind):
        ind = Individual.list_to_ind(list_ind, method, error)
        ind.calc_fitness()
        return ind.fitness,
    
    # Calcula o fitness do indivíduo simulando o sistema de ODEs
    def calc_fitness(self):
        try:
            equation = Equation(self.numerical_coeffs, labels)
            # Métodos do solve_ivp: RK45, DOP853, LSODA, BDF... 
            y = solve_ivp(self.system, t_span, initial_conditions, method=self.method, t_eval=t_eval, args=(equation, ), min_step=0.001).y
            
            if self.error == 'ABS':
                self.fitness = self.abs_error(original, y)
            elif self.error == 'MSE':
                self.fitness = self.MSE_error(original, y)
            elif self.error == 'MABS':
                self.fitness = self.mean_abs_error(original, y)
            elif self.error == 'SQUARED':
                self.fitness = self.squared_error(original, y)
            
            self.fitness = min(self.fitness, 1e6)
        except:
            # Trata exceções relacionadas ao solver
            print("Fitness Overflow")
            self.fitness = 1e6
            
    def calc_all_fitness(self):
        equation = Equation(self.numerical_coeffs, labels)
        # Métodos do solve_ivp: RK45, DOP853, LSODA, BDF... 
        y = solve_ivp(self.system, t_span, initial_conditions, method=self.method, t_eval=t_eval, args=(equation, ), min_step=0.001).y
        
        return {
            'ABS_Fitness': self.abs_error(original, y), 
            'SQUARED_Fitness': self.squared_error(original, y),
            'MSE_Fitness': self.MSE_error(original, y), 
            'MABS_Fitness': self.mean_abs_error(original, y), 
        }
     
       
    @staticmethod
    def system(t, y, equation):
        vals = [Solvers.norm_hardcoded(val, max_data[label]) for val, label in zip(y, labels)]
        N_A, N_B, N_C, N_D, N_E = vals
        
        dA = equation.complex_eqs(vals, 'A', [['-A', '-D', '-E'], ['-A', '-C', '+E'], ['+A', '+D', '-E']])
        dB = equation.complex_eqs(vals, 'B', [['-A', '-D', '-E'], ['-A', '-C', '+E'], ['+A', '+D', '-E']])
        dC = equation.complex_eqs(vals, 'C', [['+D'], ['+E']])
        dD = equation.full_eq(vals, 'D', 'C')
        dE = equation.complex_eqs(vals, 'E', [['+A', '+B', '+C'], ['+E']])

        return [dA, dB, dC, dD, dE]
    
    @staticmethod
    def abs_error(original, pred):
        return sum(sum(abs(original-pred)))
    
    @staticmethod
    def squared_error(original, pred):
        return sum(sum( (original-pred)**2 ))**(1/2)
    
    @staticmethod
    def MSE_error(original, pred):
        return np.mean((original-pred)**2)
    
    @staticmethod
    def mean_abs_error(original, pred):
        return np.mean(abs(original-pred))
            
    @staticmethod
    def initialize_ind(bounds):
        ind = Individual()
        for key, label in ind.coeffs.items():
            label['tau'] = Coefficient(bounds['tau'])
            for key, coeffs in label.items():
                if key != 'tau':
                    coeffs['n'] = Coefficient(bounds['n'])
                    coeffs['k'] = Coefficient(bounds['k'])
                    
        ind.calc_fitness()
        return ind
    
    @property
    def numerical_coeffs(self,):
        
        numerical_coeffs = copy.deepcopy(self.coeffs)
        for key, label in numerical_coeffs.items():
            label['tau'] = label['tau'].val
            for key, coeffs in label.items():
                if key != 'tau':
                    coeffs['n'] = int(coeffs['n'].val)
                    coeffs['k'] = coeffs['k'].val
                    
        return numerical_coeffs
    
    def plot(self, method='RK45', comparison=True):
        methods = [self.method]
        results = {}
        equation = Equation(self.numerical_coeffs, labels)
        # Métodos do solve_ivp: RK45, DOP853, LSODA, BDF... 
        results[method] = solve_ivp(self.system, t_span, initial_conditions, method=method, t_eval=t_eval, args=(equation, ), min_step=0.001).y
        Plotters.plot_methods(results=results,t=t_eval, methods=methods, labels=labels)
        if comparison:
            Plotters.plot_comparison(results=results, t=t_eval, df=df, methods=methods, labels=labels)
            
    @staticmethod        
    def initialize_average_bounds(bounds, ind_size):
        averages = [np.mean(value) for value in bounds.values()]
        array = np.resize(averages, ind_size)
        return array   
        
    def __repr__(self):
        coeffs_repr = {k: v for k, v in self.coeffs.items()}
        return f"Individual (fitness={self.fitness}, coeffs={coeffs_repr}, ind_size={self.ind_size})"

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

def instantiate_toolbox():
    # Criação do tipo de indivíduo e toolbox
    toolbox = base.Toolbox()

    # Inicializa estratégia CMA-ES
    centroids = Individual.initialize_average_bounds(bounds, IND_SIZE)
    strategy = cma.Strategy(centroid=centroids, sigma=10, lambda_=int(4+(3*np.log(IND_SIZE))))

    toolbox.register("generate", strategy.generate, creator.Individual)
    toolbox.register("update", strategy.update)

    # Estatísticas e parâmetros do algoritmo
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda individual: individual.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    return toolbox, hof, strategy, stats

In [ ]:
# Configuração do log
logging.basicConfig(
    filename="ECOLI_CMAES_execution.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

if os.path.exists('ECOLI_CMAES.csv'):
    results_df = pd.read_csv('ECOLI_CMAES.csv')
    existing_results = results_df[['seed', 'error_type', 'method']]
else:
    results_df = pd.DataFrame(columns=[
        'best_ind', 'error_type', 'method', 'seed', 'ABS_Fitness',
        'SQUARED_Fitness', 'MSE_Fitness', 'MABS_Fitness', 'execution_time'
    ])
    existing_results = pd.DataFrame(columns=['seed', 'error_type', 'method'])

all_methods = ['RK45', 'DOP853', 'LSODA']
errors = ['ABS', 'SQUARED', 'MSE', 'MABS']

seeds = 10
results = []

TIMEOUT_LIMIT = 60*30  # 30 min timeout

def run_evolution(seed, error, method, result_container):
    logging.info(f"Starting execution for: Solver={method}, Error={error}, Seed={seed}")
    print(f"Solver: {method}, Error: {error}")

    toolbox, hof, strategy, stats = instantiate_toolbox()
    np.random.seed(seed)
    toolbox.register("evaluate", Individual.cma_evaluate, method, error)
    population = toolbox.generate()
    Individual.apply_bounds(population, method, error)

    best_fitness = None
    no_improvement_counter = 0
    best_individual = None
    start_time = time.time()

    try:
        for gen in range(GENERATION_LIMIT):
            if time.time() - start_time > TIMEOUT_LIMIT:
                raise TimeoutError(f"Timeout reached for {method} on seed {seed}, error {error}")

            for i, ind in enumerate(population):
                ind.fitness.values = toolbox.evaluate(ind)

            record = stats.compile(population)
            current_best_fitness = min(ind.fitness.values[0] for ind in population)
            hof.update(population)

            if best_fitness is None or current_best_fitness < best_fitness - TOLERANCE:
                best_fitness = current_best_fitness
                no_improvement_counter = 0
                best_individual = hof[0]
            else:
                no_improvement_counter += 1

            if no_improvement_counter >= (NO_IMPROVEMENT_LIMIT + gen / 100):
                strategy.sigma = min(max(strategy.sigma * HARD_SIGMA_INCREASE_FACTOR, 1), 15)
                logging.info(f"Sigma increased: {strategy.sigma}")
                print(f"Sigma increased due to no improvement. New sigma: {strategy.sigma}")
                no_improvement_counter = 0

            toolbox.update(population)
            print(f"Generation {gen}: {record}", end='\r')

            if gen % 500 == 0 and best_individual:
                best_ind = Individual.list_to_ind(best_individual, method, error)
                best_ind.plot(comparison=False, method=method)

            population = toolbox.generate()
            Individual.apply_bounds(population, method, error)

    except TimeoutError as e:
        logging.warning(e)
        print(e)  # Log timeout event
    except Exception as e:
        logging.error(f"Unexpected error: {e}", exc_info=True)

    finally:
        end_time = time.time()
        execution_time = end_time - start_time

        # Se o melhor indivíduo não existir, salvar valores nulos
        errors_result = best_ind.calc_all_fitness() if best_ind else {
            'ABS_Fitness': None, 'SQUARED_Fitness': None, 'MSE_Fitness': None, 'MABS_Fitness': None
        }

        result_data = {
            'best_ind': best_ind if best_ind else None,
            'error_type': error,
            'method': method,
            'seed': seed,
            'ABS_Fitness': errors_result['ABS_Fitness'],
            'SQUARED_Fitness': errors_result['SQUARED_Fitness'],
            'MSE_Fitness': errors_result['MSE_Fitness'],
            'MABS_Fitness': errors_result['MABS_Fitness'],
            'execution_time': execution_time,
        }

        result_container.append(result_data)
        logging.info(f"Execution finished for {method}, seed {seed}, error {error} in {execution_time:.2f} seconds.")

# Main loop
for seed in range(seeds):
    for error in errors:
        for method in all_methods:
            if not existing_results[
                (existing_results['seed'] == seed) & 
                (existing_results['error_type'] == error) & 
                (existing_results['method'] == method)
            ].empty:
                logging.info(f"Skipping: seed={seed}, error={error}, method={method}")
                print(f"Skipping already processed: seed={seed}, error={error}, method={method}")
                continue

            result_container = []
            thread = threading.Thread(target=run_evolution, args=(seed, error, method, result_container))
            thread.start()
            thread.join(timeout=TIMEOUT_LIMIT)

            if thread.is_alive():
                logging.warning(f"Force-stopping {method} for seed {seed}, error {error} due to timeout.")
                print(f"Force-stopping {method} for seed {seed}, error {error} due to timeout.")

            if result_container:
                results.append(result_container[0])
                results_df = pd.concat([results_df, pd.DataFrame([result_container[0]])], ignore_index=True)
                results_df.to_csv('ECOLI_CMAES.csv', index=False)
            else:
                logging.warning(f"WARNING: No results stored for {method}, seed {seed}, error {error}.")
                print(f"WARNING: No results stored for {method}, seed {seed}, error {error}.")


In [ ]:
results_df